In [1]:
import sys

sys.path.append("../embeddings/")

from chroma_funcs import get_closest_entries

/Users/sam/miniconda3/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def format_examples(examples):
    result = "Given the following examples:\n"
    for i, example_group in enumerate(examples):
        for j, example in enumerate(example_group):
            result += f"Example {j+1}:"
            result += "\n### Input:\n"
            result += example.get('question', 'N/A') + "\n"
            result += "\n### Context:\n"
            result += example.get('context', 'N/A') + "\n"
            result += "\n### Response:\n"
            result += example.get('answer', 'N/A') + "\n\n"
        if i < len(examples) - 1:
            result += "\n"
    return result


def generate_rag_sql_prompt(data_point, dataset_name):
    results = get_closest_entries(
        dataset_name,
        "question",
        data_point["question"],
        n_results=5,
        dataset_split="train",
    )

    rag_datapoints = results["metadatas"]
    # print("rag datapoints are: ", rag_datapoints)
    formatted_examples = format_examples(rag_datapoints)
    # print("formatted examples are: ", formatted_examples)
    inference_prompt = f"""You are a powerful text-to-SQL model. Your job is to answer questions about a database. You are given a question and context regarding one or more tables. You must output the SQL query that answers the question.

{formatted_examples}Please generate the SQL query that answers the following:
### Input:
{data_point["question"]}

### Context:
{data_point["context"]}

### Response:
"""
    full_prompt = f"{inference_prompt}\n{data_point['answer']}"
    return full_prompt, inference_prompt

In [43]:
test_datapoint = {
        "question": "What is the average horsepower for all cars produced before 1980 ?",
        "context": "CREATE TABLE cars_data (horsepower INTEGER, year INTEGER)",
        "answer": "select avg(horsepower) from cars_data where year  <  1980;",
        "db_id": "car_1"
    }

print(generate_rag_sql_prompt(test_datapoint, "samlhuillier/sql-create-context-spider-intersect"))

collection.count() 6602
collection.count() 6602
[[{'answer': 'SELECT AVG(Working_Horses) FROM farm WHERE Total_Horses > 5000', 'context': 'CREATE TABLE farm (Working_Horses INTEGER, Total_Horses INTEGER)', 'question': 'What is the average number of working horses of farms with more than 5000 total number of horses?'}, {'answer': 'SELECT AVG(Working_Horses) FROM farm WHERE Total_Horses > 5000', 'context': 'CREATE TABLE farm (Working_Horses INTEGER, Total_Horses INTEGER)', 'question': 'What is the average number of working horses of farms with more than 5000 total number of horses?'}, {'answer': 'SELECT AVG(Price), AVG(Cases) FROM WINE WHERE YEAR = 2009 AND Grape = "Zinfandel"', 'context': 'CREATE TABLE WINE (Price INTEGER, Cases INTEGER, YEAR VARCHAR, Grape VARCHAR)', 'question': 'What are the average prices and cases of wines produced in the year of 2009 and made of Zinfandel grape?'}, {'answer': 'SELECT AVG(Price), AVG(Cases) FROM WINE WHERE YEAR = 2009 AND Grape = "Zinfandel"', 'cont

In [3]:
from datasets import load_dataset

dataset = load_dataset("samlhuillier/sql-create-context-spider-intersect", split="validation")

def add_prompt_features(example):
    # Add your logic to generate the extra feature here
    full_prompt, inference_prompt = generate_rag_sql_prompt(example, "samlhuillier/sql-create-context-spider-intersect")
    example['full_prompt'] = full_prompt
    example['inference_prompt'] = inference_prompt
    return example

dataset = dataset.map(add_prompt_features)

# Step 4: Save the dataset as a JSON file
dataset.to_json('sql-create-context-spider-intersect-validation-with-prompts.jsonl')
# prompts = []
# for data_point in dataset:
#     prompts.append(generate_rag_sql_prompt(data_point, "samlhuillier/sql-create-context-spider-intersect"))
#     # print(generate_rag_sql_prompt(data_point, "samlhuillier/sql-create-context-spider-intersect"))

# print(prompts[8])
# prompted_dataset = dataset.map(
#     lambda x: generate_rag_sql_prompt(x, "samlhuillier/sql-create-context-spider-intersect")
# )
# prompted_dataset
    

Creating json from Arrow format: 100%|██████████| 1/1 [00:00<00:00, 15.91ba/s]


4106046

In [13]:
import json

# with open("sql-create-context-spider-intersect-validation-with-prompts.jsonl", "r") as f:
#     data = json.load(f)

# with open('sql-create-context-spider-intersect-validation-with-prompts.jsonl', 'r') as f:
#     for line in f:
#         data = json.loads(line)
#         # print(data)

data = []
with open('sql-create-context-spider-intersect-validation-with-prompts.jsonl', 'r') as f:
    for line in f:
        data.append(json.loads(line))
# print(data_list[2])
# Step 2: Apply the function to each entry
# responses = map(your_function, data)
prompts = []
for data_point in data:
    # prompt = generate_prompt(data_point)
    # append prompt to prompts
    prompts.append(data_point['inference_prompt'])

prompts[3]

'You are a powerful text-to-SQL model. Your job is to answer questions about a database. You are given a question and context regarding one or more tables. You must output the SQL query that answers the question.\n\nGiven the following examples:\nExample 1:\n### Input:\nList the age of all music artists.\n\n### Context:\nCREATE TABLE artist (Age VARCHAR)\n\n### Response:\nSELECT Age FROM artist\n\nExample 2:\n### Input:\nList the age of all music artists.\n\n### Context:\nCREATE TABLE artist (Age VARCHAR)\n\n### Response:\nSELECT Age FROM artist\n\nExample 3:\n### Input:\nPlease list the age and famous title of artists in descending order of age.\n\n### Context:\nCREATE TABLE artist (Famous_Title VARCHAR, Age VARCHAR)\n\n### Response:\nSELECT Famous_Title, Age FROM artist ORDER BY Age DESC\n\nExample 4:\n### Input:\nPlease list the age and famous title of artists in descending order of age.\n\n### Context:\nCREATE TABLE artist (Famous_Title VARCHAR, Age VARCHAR)\n\n### Response:\nSELEC

In [ ]:
import torch
from transformers import AutoModelForCausalLM, BitsAndBytesConfig, AutoTokenizer

base_model = "codellama/CodeLlama-7b-hf"
model = AutoModelForCausalLM.from_pretrained(
    base_model,
    load_in_8bit=True,
    torch_dtype=torch.float16,
    device_map="auto",
)
tokenizer = AutoTokenizer.from_pretrained("codellama/CodeLlama-7b-hf")

In [ ]:
from peft import PeftModel

model = PeftModel.from_pretrained(model, "sql-code-llama/checkpoint-160")

In [ ]:
tokenizer.pad_token = tokenizer.eos_token

In [ ]:
batch_size = 12  # You can adjust the batch size based on your GPU capacity
outputs = []

model.eval()
with torch.no_grad(), open("codellama7b-valoutputs2.txt", "a") as f:
    for i in range(0, len(prompts), batch_size):
        print("i is: ", i)
        batch_inputs = prompts[i : i + batch_size]

        # Step 3: Loop over the batches and generate the outputs
        batch_model_inputs = tokenizer(
            batch_inputs, return_tensors="pt", padding=True, truncation=True
        )

        input_lengths = [
            len(input_ids) for input_ids in batch_model_inputs["input_ids"]
        ]

        batch_outputs = model.generate(**batch_model_inputs, max_new_tokens=100)

        # Step 4: Collect the outputs
        batch_decoded_outputs = [
            tokenizer.decode(output[input_length:], skip_special_tokens=True)
            for output, input_length in zip(batch_outputs, input_lengths)
        ]
        outputs.extend(batch_decoded_outputs)
        print(outputs)
        for b in batch_decoded_outputs:
            f.write(b.replace("\n", " ") + "\n")